In [1]:
# imports y rutas
import pandas as pd
import numpy as np
import re
from pathlib import Path

log_file_path = Path(r"C:\Users\Jose Ruiz\Desktop\tripadvisor_output\Full\proceso.log")
def log_step(msg):
    log_line = f"[STEP] {msg}"
    # Mostrar en consola
    print(log_line)
    # Guardar en archivo .txt (append)
    with open(log_file_path, "a", encoding="utf-8") as f:
        f.write(log_line + "\n")
        
log_step("Importar librerías, definir rutas y cargar datasets de atracciones y hoteles; luego unirlos en un único DataFrame")

# Definimos la carpeta base
base_path = Path(r"C:\Users\Jose Ruiz\Desktop\tripadvisor_output\Full")

# Definimos los archivos
file1 = base_path / "atracciones_venezuela_tripadvisor.csv"
file2 = base_path / "hoteles_venezuela_tripadvisor.csv"

# Cargamos los dos datasets
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

print("Forma df1:", df1.shape)
print("Forma df2:", df2.shape)

# Unimos ambos datasets
df = pd.concat([df1, df2], ignore_index=True)

print("Forma total unida:", df.shape)
df.head()

[STEP] Importar librerías, definir rutas y cargar datasets de atracciones y hoteles; luego unirlos en un único DataFrame
Forma df1: (26012, 9)
Forma df2: (41950, 9)
Forma total unida: (67962, 9)


,Location Name,Lugar,Text,Rating,Year,Month,Day,Ciudad,Tipo
0,Angel_Falls,Angel without a helicopter is not Angel,"Pegasus tour costs $750 per person, helicopter...",5,2025,3,11,Canaima_National_Park_Guayana_Region,atraccion
1,Angel_Falls,Bra stolen,I as a man always wear a bra and I wanted to s...,1,2025,2,6,Canaima_National_Park_Guayana_Region,atraccion
2,Angel_Falls,A 4 x 4 experience,This excursion I think is the one that everyon...,5,2024,9,11,Canaima_National_Park_Guayana_Region,atraccion
3,Angel_Falls,A dream come true!!!,"One of the best adventures of my life, I met h...",5,2024,8,29,Canaima_National_Park_Guayana_Region,atraccion
4,Angel_Falls,Unforgettable experience and very good organiz...,My trip to Canaima and Salto Angel was unforge...,5,2024,6,25,Canaima_National_Park_Guayana_Region,atraccion


Lee el CSV y asegura tipos; evita que cadenas como “NA” se conviertan en NaN accidentalmente y reporta el shape inicial.

In [2]:
log_step("Convertir columnas del DataFrame a los tipos de datos definidos en dtype_dict")


# Diccionario de tipos deseados
dtype_dict = {
    "Location Name": "string",
    "Lugar": "string",
    "Text": "string",
    "Rating": "Int64",   # Entero que acepta NaN
    "Year": "Int64",
    "Month": "Int64",
    "Day": "Int64",
    "Ciudad": "string",
    "Tipo": "string",
}

# Convertir los tipos después de tener df ya unido
df = df.astype(dtype_dict)

print("Filas x columnas (con tipos corregidos):", df.shape)
df.info()
df.head(3)

[STEP] Convertir columnas del DataFrame a los tipos de datos definidos en dtype_dict
Filas x columnas (con tipos corregidos): (67962, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67962 entries, 0 to 67961
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Location Name  67962 non-null  string
 1   Lugar          67920 non-null  string
 2   Text           67962 non-null  string
 3   Rating         67962 non-null  Int64 
 4   Year           67962 non-null  Int64 
 5   Month          67962 non-null  Int64 
 6   Day            67962 non-null  Int64 
 7   Ciudad         67962 non-null  string
 8   Tipo           67962 non-null  string
dtypes: Int64(4), string(5)
memory usage: 4.9 MB


,Location Name,Lugar,Text,Rating,Year,Month,Day,Ciudad,Tipo
0,Angel_Falls,Angel without a helicopter is not Angel,"Pegasus tour costs $750 per person, helicopter...",5,2025,3,11,Canaima_National_Park_Guayana_Region,atraccion
1,Angel_Falls,Bra stolen,I as a man always wear a bra and I wanted to s...,1,2025,2,6,Canaima_National_Park_Guayana_Region,atraccion
2,Angel_Falls,A 4 x 4 experience,This excursion I think is the one that everyon...,5,2024,9,11,Canaima_National_Park_Guayana_Region,atraccion


Renombrar columnas a un esquema claro

Qué hace: Renombra a nombres consistentes y en español. Mantiene el slug original y crea campos “limpios” más legibles.

In [3]:
log_step("Renombrar columnas del DataFrame a un formato estandarizado en minúsculas")
# renombrar columnas
df = df.rename(columns={
    "Location Name": "location_name",  # p.ej. 
    "Lugar": "review_title",
    "Text": "texto",
    "Rating": "rating",
    "Year": "year",
    "Month": "month",
    "Day": "day",
    "Ciudad": "ciudad_raw",
    "Tipo": "tipo"
})

[STEP] Renombrar columnas del DataFrame a un formato estandarizado en minúsculas


Normalizar y limpiar strings

Los textos suelen traer espacios extras, saltos de línea o dobles espacios. Este paso limpia todas las columnas de tipo texto.

In [4]:
log_step("Limpiar columnas de texto: reemplazar múltiples espacios, eliminar espacios sobrantes y rellenar NaN con vacío")
# limpieza de strings
str_cols = df.select_dtypes(include="object").columns
for c in str_cols:
    df[c] = (df[c]
             .fillna("")                       # evita errores con NaN
             .str.replace(r"\s+", " ", regex=True)  # colapsa espacios múltiples
             .str.strip())                     # quita espacios inicio/fin

[STEP] Limpiar columnas de texto: reemplazar múltiples espacios, eliminar espacios sobrantes y rellenar NaN con vacío


Construir columna de fecha

Usamos las columnas year, month y day para crear un campo fecha válido.

In [5]:
log_step("Crear columna de fecha a partir de year, month y day; eliminar registros con fechas inválidas")
# crear columna de fecha
df["fecha"] = pd.to_datetime(
    dict(year=df["year"], month=df["month"], day=df["day"]),
    errors="coerce"  # si la fecha es inválida -> NaT
)

# Eliminar registros con fechas inválidas
df = df[df["fecha"].notna()].copy()

[STEP] Crear columna de fecha a partir de year, month y day; eliminar registros con fechas inválidas


Validar y filtrar ratings

Nos aseguramos que rating esté entre 1 y 5

In [6]:
log_step("Filtrar únicamente registros con ratings válidos entre 1 y 5")
# ratings válidos
df = df[df["rating"].between(1, 5, inclusive="both")].copy()

[STEP] Filtrar únicamente registros con ratings válidos entre 1 y 5


Crear un nombre legible para location_name

Convertimos el location_name (que viene con guiones bajos) en un formato bonito.

In [7]:
log_step("Normalizar location_name convirtiendo slugs con guiones bajos a títulos capitalizados en nueva columna 'location'")
# limpiar location_name
def slug_to_title(s):
    if not s:
        return None
    return " ".join([p.capitalize() for p in str(s).split("_") if p])

df["location"] = df["location_name"].apply(slug_to_title)

[STEP] Normalizar location_name convirtiendo slugs con guiones bajos a títulos capitalizados en nueva columna 'location'


Separar ciudad y región de ciudad_raw

La columna ciudad_raw tiene formatos tipo Margarita_island_coastal_island_insular_region.

In [8]:
log_step("Parsear columna ciudad_raw para separar en 'ciudad' y 'region'")
# parsear ciudad y región
def split_ciudad(c_raw):
    if not c_raw:
        return pd.Series([None, None])
    parts = c_raw.split("_")
    if len(parts) >= 2 and parts[-1].lower() == "region":
        region = f"{parts[-2].capitalize()} {parts[-1].capitalize()}"
        ciudad = " ".join(p.capitalize() for p in parts[:-2])
    else:
        region = None
        ciudad = " ".join(p.capitalize() for p in parts)
    return pd.Series([ciudad.strip(), region.strip() if region else None])

df[["ciudad", "region"]] = df["ciudad_raw"].apply(split_ciudad)

[STEP] Parsear columna ciudad_raw para separar en 'ciudad' y 'region'


Limpiar texto y generar una versión “texto_limpio”

Quitamos URLs, saltos de línea, HTML y espacios innecesarios.

In [9]:
log_step("Limpiar texto de reviews eliminando URLs, etiquetas HTML, saltos de línea y espacios extra; crear columna 'texto_limpio'")
# limpieza de texto

URL_RE = re.compile(r"https?://\S+|www\.\S+")
TAG_RE = re.compile(r"<.*?>")

def clean_text(t):
    if not isinstance(t, str):
        return None
    t = URL_RE.sub(" ", t)
    t = TAG_RE.sub(" ", t)
    t = t.replace("\n", " ").replace("\r", " ")
    t = re.sub(r"\s+", " ", t).strip()
    return t

df["texto_limpio"] = df["texto"].apply(clean_text)
df = df.drop(columns='texto')

[STEP] Limpiar texto de reviews eliminando URLs, etiquetas HTML, saltos de línea y espacios extra; crear columna 'texto_limpio'


Deduplicar

Quitamos duplicados exactos (mismo texto + mismo lugar).

In [10]:
log_step("Eliminar duplicados basados en location_name y texto_limpio, conservando la primera ocurrencia")
# Pdeduplicar por location_name + texto_limpio
before = len(df)
df = df.drop_duplicates(subset=["location_name", "texto_limpio"], keep="first")
print(f"Eliminadas {before - len(df)} filas duplicadas.")

[STEP] Eliminar duplicados basados en location_name y texto_limpio, conservando la primera ocurrencia
Eliminadas 22 filas duplicadas.


In [11]:
#comprobar nulos
print(df.isna().sum())

location_name       0
review_title       42
rating              0
year                0
month               0
day                 0
ciudad_raw          0
tipo                0
fecha               0
location            0
ciudad              0
region           6226
texto_limpio        0
dtype: int64


In [12]:
log_step("Rellenar valores nulos en 'region' con 'Unknown' y en 'review_title' con 'Generic title'")
# Rellenar valores nulos
df["region"] = df["region"].fillna("Unknown")
df["review_title"] = df["review_title"].fillna("Generic title")

# Verificamos nuevamente
print(df.isna().sum())

[STEP] Rellenar valores nulos en 'region' con 'Unknown' y en 'review_title' con 'Generic title'
location_name    0
review_title     0
rating           0
year             0
month            0
day              0
ciudad_raw       0
tipo             0
fecha            0
location         0
ciudad           0
region           0
texto_limpio     0
dtype: int64


Reordenar columnas finales

Dejamos un dataset limpio y ordenado.

In [13]:
log_step("Reordenar columnas del DataFrame según el esquema final definido en final_cols")
# reordenar columnas
final_cols = [
    "location_name", "location",
    "review_title", "texto_limpio",
    "rating", "fecha", "year", "month", "day",
    "ciudad_raw", "ciudad", "region",
    "tipo"
]
df = df[final_cols]
df.head()

[STEP] Reordenar columnas del DataFrame según el esquema final definido en final_cols


,location_name,location,review_title,texto_limpio,rating,fecha,year,month,day,ciudad_raw,ciudad,region,tipo
0,Angel_Falls,Angel Falls,Angel without a helicopter is not Angel,"Pegasus tour costs $750 per person, helicopter...",5,2025-03-11,2025,3,11,Canaima_National_Park_Guayana_Region,Canaima National Park,Guayana Region,atraccion
1,Angel_Falls,Angel Falls,Bra stolen,I as a man always wear a bra and I wanted to s...,1,2025-02-06,2025,2,6,Canaima_National_Park_Guayana_Region,Canaima National Park,Guayana Region,atraccion
2,Angel_Falls,Angel Falls,A 4 x 4 experience,This excursion I think is the one that everyon...,5,2024-09-11,2024,9,11,Canaima_National_Park_Guayana_Region,Canaima National Park,Guayana Region,atraccion
3,Angel_Falls,Angel Falls,A dream come true!!!,"One of the best adventures of my life, I met h...",5,2024-08-29,2024,8,29,Canaima_National_Park_Guayana_Region,Canaima National Park,Guayana Region,atraccion
4,Angel_Falls,Angel Falls,Unforgettable experience and very good organiz...,My trip to Canaima and Salto Angel was unforge...,5,2024-06-25,2024,6,25,Canaima_National_Park_Guayana_Region,Canaima National Park,Guayana Region,atraccion


In [14]:
print(df["ciudad"].unique())
print(df["ciudad_raw"].unique())

['Canaima National Park' 'Caracas' 'Margarita Island Coastal Islands'
 'Maracaibo' 'Los Roques National Park Coastal Islands'
 'Chichiriviche Morrocoy National Park Central' 'Merida'
 'Coche Island Coastal Islands'
 'Pampatar Margarita Island Coastal Islands'
 'Juan Griego Margarita Island Coastal Islands' 'Puerto Cabello'
 'Ciudad Guayana'
 'Isla El Gran Roque Los Roques National Park Coastal Islands' 'North'
 'Porlamar Margarita Island Coastal Islands' 'Tucacas Central' ''
 'Cumana North' 'San Cristobal' 'Choroni' 'Canaima'
 'El Yaque Margarita Island Coastal Islands Insular'
 'Isla El Gran Roque Los Roques National Park Coastal Islands Insu'
 'Pampatar Margarita Island Coastal Islands Insular' 'El Tigre'
 'Maiquetia' 'Maracay' 'Barinas' 'Coro Central' 'Lecheria North'
 'Acarigua Central' 'Playa El Agua Margarita Island Coastal Islands'
 'Naguanagua' 'El Yaque Margarita Island Coastal Islands' 'Valencia'
 'San Pedro Coche Island Coastal Islands' 'El Vigia'
 'Gran Roque Isla El Gran R

In [15]:
log_step("Normalizar 'ciudad_raw' y unificar nombres de ciudades (Los Roques, Isla Margarita, Isla de Coche, Morrocoy, etc.) en columna 'ciudad'")

# --- Limpieza directa de 'ciudad_raw' ---
def clean_ciudad_raw(s):
    if not isinstance(s, str):
        return s
    s = s.strip()
    # Normalizamos sufijos truncados de región
    s = re.sub(r"Insu.*Region$", "Insular_Region", s, flags=re.I)
    s = re.sub(r"North.*Region$", "North_Eastern_Region", s, flags=re.I)
    s = re.sub(r"Central.*Region$", "Central_Western_Region", s, flags=re.I)
    return s

df["ciudad_raw"] = df["ciudad_raw"].apply(clean_ciudad_raw)

# --- Limpieza directa de 'ciudad' ---
def clean_ciudad(s):
    if not isinstance(s, str):
        return s
    
    s_low = s.lower()

    # Unificamos Los Roques
    if "los roques" in s_low or "gran roque" in s_low or "el roque" in s_low:
        return "Los Roques"

    # Unificamos Margarita
    if ("margarita" in s_low or "pampatar" in s_low or "porlamar" in s_low or 
        "juan griego" in s_low or "playa el agua" in s_low or "el yaque" in s_low or 
        "el tirano" in s_low or "arismendi" in s_low or "pedro gonzales" in s_low):
        return "Isla Margarita"

    # Unificamos Coche
    if "coche" in s_low:
        return "Isla de Coche"

    # Unificamos Morrocoy
    if "morrocoy" in s_low or "chichiriviche" in s_low or "tucacas" in s_low:
        return "Morrocoy"

    # Si no coincide con ninguna regla, devolvemos el valor original capitalizado
    return s.title()

df["ciudad"] = df["ciudad"].apply(clean_ciudad)

[STEP] Normalizar 'ciudad_raw' y unificar nombres de ciudades (Los Roques, Isla Margarita, Isla de Coche, Morrocoy, etc.) en columna 'ciudad'


In [16]:
print(df["ciudad"].unique())
print(df["ciudad_raw"].unique())

['Canaima National Park' 'Caracas' 'Isla Margarita' 'Maracaibo'
 'Los Roques' 'Morrocoy' 'Merida' 'Isla de Coche' 'Puerto Cabello'
 'Ciudad Guayana' 'North' '' 'Cumana North' 'San Cristobal' 'Choroni'
 'Canaima' 'El Tigre' 'Maiquetia' 'Maracay' 'Barinas' 'Coro Central'
 'Lecheria North' 'Acarigua Central' 'Naguanagua' 'Valencia' 'El Vigia'
 'Barquisimeto Central' 'Punto Fijo Paraguana Peninsula Central'
 'Tucupita Orinoco Delta' 'Ocumare De La Costa' 'La Colonia Tovar'
 'Bejuma' 'El Cobre' 'Upata' 'Caruao' 'Playa Colorada North' 'Osma']
['Canaima_National_Park_Guayana_Region' 'Caracas_Capital_Region'
 'Margarita_Island_Coastal_Islands_Insular_Region'
 'Maracaibo_Zulian_Region'
 'Los_Roques_National_Park_Coastal_Islands_Insular_Region'
 'Chichiriviche_Morrocoy_National_Park_Central_Western_Region'
 'Merida_Andean_Region' 'Coche_Island_Coastal_Islands_Insular_Region'
 'Pampatar_Margarita_Island_Coastal_Islands_Insular_Region'
 'Juan_Griego_Margarita_Island_Coastal_Islands_Insular_Region'

In [17]:
log_step("Guardar DataFrame final como CSV 'tripadvisor_guacamaia_masterbase_ve.csv'")

output_path = Path(r"C:\Users\Jose Ruiz\Desktop\tripadvisor_output\Full\final_masterbase")

# ===  Guardar dataset ===
final_file = output_path / "tripadvisor_guacamaia_masterbase_ve.csv"
df.to_csv(final_file, index=False, encoding="utf-8-sig")

print(f"Archivo guardado en: {final_file}")

[STEP] Guardar DataFrame final como CSV 'tripadvisor_guacamaia_masterbase_ve.csv'
Archivo guardado en: C:\Users\Jose Ruiz\Desktop\tripadvisor_output\Full\final_masterbase\tripadvisor_guacamaia_masterbase_ve.csv
